# Clasificación de margen máximo con máquinas de vectores de soporte#
Otro algoritmo de aprendizaje potente y ampliamente utilizado es la **máquina de vectores de soporte (SVM)**, que puede considerarse una extensión del perceptrón. Al utilizar el algoritmo perceptron, se minimizan los errores de clasificación errónea. Sin embargo, en SVM el objetivo de optimización es maximizar el margen. El **margen** se define como la distancia entre el hiperplano de separación (límite de decisión) y los ejemplos de entrenamiento más cercanos a este hiperplano, que son los llamados **vectores de soporte**.

![Margenes maximizados](imgs/maxmargin.png)

### Intuición sobre el margen máximo ###
La razón detras de tener límites de decisión con márgenes grandes es que tienden a tener menores errores de generalización, mientras que los modelos con márgenes pequeños son más propensos al sobreajuste. Para tener una idea sobre la maximización de margenes, nos vamos a centrar esos hiperplanos positivo y negativo paralelos a el límite de decisión, que se pueden expresar como:

$$\begin{equation*} w_0 + \mathbf{w}^T\mathbf{x}_{pos}=1\end{equation*}$$
$$\begin{equation*} w_0 + \mathbf{w}^T\mathbf{x}_{neg}=-1\end{equation*}$$

Si restamos estas dos ecuaciones, se tiene:

$$\Rightarrow \mathbf{w}^T \big(\mathbf{x}_{pos} - \mathbf{x}_{new} \big) = 2 $$

Se pude normalizar la ecuación mediante la norma o longitud del vector $w$, que se define como:

$$ \begin{Vmatrix}\mathbf{w}\end{Vmatrix} = \sqrt{\sum_{j=1}^m w_j^2}$$

Así se llega a la siguiente equación:

$$\begin{equation*}\frac{ \mathbf{w}^T \big(\mathbf{x}_{pos} - \mathbf{x}_{neg} \big)}{\begin{Vmatrix}\mathbf{w}\end{Vmatrix}} = \frac{2}{\begin{Vmatrix}\mathbf{w}\end{Vmatrix}} \end{equation*}$$

La exprsión de la izquierda se puede interpretar como la distancia entre el hiperploano positivo y negativo, que es lo que se denomina **margen** y se quiere maximizar.

Ahora, la función objetivo de la SVM se convierte en la maximización de este margen maximizando $\frac{2}{\begin{Vmatrix}\mathbf{w}\end{Vmatrix}}$ bajo la restrición de que los ejemplos se han clasificado correctamente, que se puede escribir como:

$$\begin{equation*}w_0 + \mathbf{w}^T\mathbf{x}^{(i)} \geq 1\,\, si y^{(i)}=1 \end{equation*}$$

$$\begin{equation*}w_0 + \mathbf{w}^T\mathbf{x}^{(i)} \leq -1\,\, si y^{(i)}=-1\end{equation*}$$

$$ para\,i = 1...N$$

Donde $N$ es el número de ejemplos en el conjunto de datos.

Estas dos ecuaciones básicamente dicen que todos los ejemplos de clase negativa deben caer en el lado negativo del hiperplano negativo, mientras que los ejemplos de clase positiva caeran en el lado positivo del hiperplano positivo, lo cual se puede escribri de forma más compacta como sigue:

$$\begin{equation}y^{(i)}\big(w_0 + \mathbf{w}^T\mathbf{x}^{(i)}\big) \geq 1\,\, \forall i\end{equation}$$

En la práctica es más sencillo minimizar el termino reciproco, $\frac{1}{2}\begin{Vmatrix}\mathbf{w}\end{Vmatrix}^2$, que se puede resolver mediante programación cuadrática